\#SusanRice, continued
-----------------

Today we are going to continue our work on the Susan Rice story. We have pulled all the tweets containing #SusanRice and packaged them up for you. There is so much to dig in here. 

[http://compute-cuj.org/tweets.csv](http://compute-cuj.org/tweets.csv)

[http://compute-cuj.org/media.csv](http://compute-cuj.org/media.csv)

[http://compute-cuj.org/hashtags.csv](http://compute-cuj.org/hashtags.csv)

[http://compute-cuj.org/urls.csv](http://compute-cuj.org/urls.csv)

[http://compute-cuj.org/mentions.csv](http://compute-cuj.org/mentions.csv)

And here are the tweets.

[http://compute-cuj.org/rice_tweets.tar.gz](http://compute-cuj.org/rice_tweets.tar.gz)


There are two references we might want to consider. The first involves networking... how might we apply this here?
[Pew report on mapping.](http://www.pewinternet.org/2014/02/20/mapping-twitter-topic-networks-from-polarized-crowds-to-community-clusters/) And the other has to do with ad trackers. It points to case studies and nice tools to see what kind of ad tracking are the different sites using.  [Tracker Tracker Tool.](https://wiki.digitalmethods.net/Dmi/WorkshopTrackingtheTrackers). Oh and another paper on tracking shared analytics tags (not just services but actual identifiers). [Some tips.](bellingcat.com/resources/how-tos/2015/07/23/unveiling-hidden-connections-with-google-analytics-ids/)


OK down to work! Here is what we did to create the files you have.

In [ ]:
from os import listdir
from json import loads

folder = "json_tweets/"
tweet_folder = listdir(folder)
len(tweet_folder)

In [ ]:
#from csv import writer

#tweets = writer(open("tweets.csv","wb"))
#hashtags = writer(open("hashtags.csv","wb"))
#urls = writer(open("urls.csv","wb"))
#mentions = writer(open("mentions.csv","wb"))
#media = writer(open("media.csv","wb"))

#for tweet_file in tweet_folder:
#    
#    tweet = loads(open(folder+tweet_file).read())
#    
#    if tweet['source']:
#        ts = tweet['source'].encode("utf-8")
#    else: ts = ""
#        
#    if tweet['in_reply_to_screen_name']:
#        irt = tweet['in_reply_to_screen_name'].encode("utf-8")
#    else: irt = ""
#        
#    out = [tweet["id"],tweet["text"].encode("utf-8"),tweet["created_at"],tweet['retweeted'],
#           irt, tweet['in_reply_to_status_id'],ts,
#           tweet["retweet_count"],tweet["user"]["screen_name"].encode("utf-8"),
#           tweet["user"]["followers_count"],tweet["user"]["friends_count"]]
#    
#    tweets.writerow(out)
#    
#    if 'entities'in tweet and 'hashtags' in tweet['entities']:
#        save = []
#        for o in tweet['entities']['hashtags']:
#        
#            if not o['text'].encode('utf-8') in save:
#                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'].encode('utf-8'),o['text'].encode('utf-8')]
#                hashtags.writerow(out)
#                save.append(o['text'].encode('utf-8'))
#    
#    if 'entities'in tweet and 'urls' in tweet['entities']:
#
#        save = []
#        for o in tweet['entities']['urls']:
#        
#            if not o['expanded_url'].encode('utf-8') in save:
#                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'].encode('utf-8'),o['expanded_url'].encode('utf-8')]
#                urls.writerow(out)
#                save.append(o['expanded_url'].encode('utf-8'))
#                
#    if 'entities'in tweet and 'user_mentions' in tweet['entities']:
#        save = []        
#        for o in tweet['entities']['user_mentions']:
#            if not o['screen_name'].encode('utf-8') in save:
#                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'].encode('utf-8'),o['screen_name'].encode('utf-8')]
#                mentions.writerow(out)
#                save.append(o['screen_name'].encode('utf-8'))
#
#    if 'entities'in tweet and 'media' in tweet['entities']:
#        save = []
#        for o in tweet['entities']['media']:
# 
#            if not o['expanded_url'].encode('utf-8') in save:
#                out = [tweet["id"],tweet["created_at"],tweet["user"]['screen_name'].encode('utf-8'),o['expanded_url'].encode('utf-8')]
#                media.writerow(out)
#                save.append(o['expanded_url'].encode('utf-8'))
#

Let's read in what we've got. It's a data frame so our lives will be easier.

In [ ]:
from pandas import read_csv

df = read_csv("tweets.csv")

In [ ]:
df["created_at"][:10]

These will make our displays a little easier to read.

In [ ]:
from pandas import to_datetime, TimeGrouper, set_option
set_option("display.max_rows",100)
set_option("display.max_colwidth",140)

Remember that we need to transform date strings into date objects. Here we do it in the quickest way possible (it takes a long long time if you don't supply the date format and pandas has to guess instead.

In [ ]:
df["stamp"] = to_datetime(df["created_at"],format='%a %b %d %H:%M:%S +0000 %Y')

df.set_index("stamp",inplace=True)
df.head()

Here is the code we used last time to give us a sense of the time series. What other things can we do here?

In [ ]:
counts = df.groupby(TimeGrouper(freq='30min')).agg({"text":{"counts":"count"}})
counts.columns = counts.columns.droplevel()
counts = counts.reset_index()

df.reset_index(inplace=True)

counts.head()

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("cocteautt","9psj3t57ti")

mydata = [go.Scatter(x=counts["stamp"],y=counts["counts"],mode="line")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Here we look at screen names. Who tweeted the most using this hashtag?

In [ ]:
df["screen_name"].value_counts()

Perhaps the better computation tabulates users by time period. 

In [ ]:
df.set_index("stamp",inplace=True)

usrcounts = df.groupby([TimeGrouper(freq='15min'),"screen_name"]).agg({"screen_name":{"counts":"count"}})
usrcounts.columns = usrcounts.columns.droplevel()
usrcounts = usrcounts.reset_index()

df.reset_index(inplace=True)

usrcounts.head(50)

In [ ]:
usrcounts.sort_values(by="counts",ascending=False)

We have created special data frames for hashtags, media items, url's and mentions. Here are the hashtags...

In [ ]:
hashtags = read_csv("hashtags.csv")
hashtags["stamp"] = to_datetime(hashtags["created_at"],format='%a %b %d %H:%M:%S +0000 %Y')

hashtags.head()

In [ ]:
hashtags["hashtag"].str.lower().value_counts()

...and the URLs...

In [ ]:
urls = read_csv("urls.csv")
urls["stamp"] = to_datetime(urls["created_at"],format='%a %b %d %H:%M:%S +0000 %Y')

urls.head()

One obvious alteration would be to pull out just the domains from the URLs. What sites are involved with the Susan Rice hashtag?

In [ ]:
urls["domain"] =urls["url"].str.replace(".*//([^/]+).*","\\1")

In [ ]:
urls["domain"].value_counts().head(100)

One troubling observation is that bit.ly links didn't seem to get expanded. How can we fix that?

In [ ]:
urls[urls["domain"]=="bit.ly"].sample(10)

In [ ]:
urls[urls["domain"]=="bit.ly"]["url"].value_counts()

Here are the people mentioned in each tweet. What might we do with this?

In [ ]:
mentions = read_csv("mentions.csv")
mentions["stamp"] = to_datetime(mentions["created_at"],format='%a %b %d %H:%M:%S +0000 %Y')

mentions.head()

In [ ]:
mentions.shape

In [ ]:
mentions["mention"].value_counts()

Here we look at a time series of mentions of "Cernovich", the person who started the story.

In [ ]:
cer = mentions[mentions["mention"]=="Cernovich"]

cer.set_index("stamp",inplace=True)

cercounts = cer.groupby(TimeGrouper(freq='30min')).agg({"mention":{"counts":"count"}})
cercounts.columns = cercounts.columns.droplevel()
cercounts = cercounts.reset_index()

cer.reset_index(inplace=True)

cercounts.head()

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("cocteautt","9psj3t57ti")

mydata = [go.Scatter(x=counts["stamp"],y=counts["counts"],mode="line")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

And here we show you how to overlay two plots (or n plots) with separate axes.

In [ ]:
cer = mentions[mentions["mention"]=="Cernovich"]

cer.set_index("stamp",inplace=True)

cercounts = cer.groupby(TimeGrouper(freq='15min')).agg({"mention":{"counts":"count"}})
cercounts.columns = cercounts.columns.droplevel()
cercounts = cercounts.reset_index()

cer.reset_index(inplace=True)

In [ ]:
df.set_index("stamp",inplace=True)

counts = df.groupby(TimeGrouper(freq='15min')).agg({"text":{"counts":"count"}})
counts.columns = counts.columns.droplevel()
counts = counts.reset_index()

df.reset_index(inplace=True)

In [ ]:
trace1 = go.Scatter(
    x=counts["stamp"],
    y=counts["counts"],
    name='total tweets'
)
trace2 = go.Scatter(
    x=cercounts["stamp"],
    y=cercounts["counts"],
    name='cernovich mentions',
    yaxis='y2'
)
mydata = [trace1, trace2]

mylayout = go.Layout(
    title='Comparing series',
    yaxis=dict(
        title='total tweets'
    ),
    yaxis2=dict(
        title='cernovice mentions',
        overlaying='y',
        side='right'
    ),
    autosize=False,width=1000,height=500
)

myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Medium-sized API pulls
-----------------
Here's  some guidance about how you might use multiple credentials to make a big pull like we did for Susan Rice.
First, load up your credetials!

In [2]:
# before we can make Twitter API calls, we need to initialize a few things...
from tweepy import OAuthHandler, API

In [5]:
CONSUMER_KEY = ""
CONSUMER_SECRET = "8X0i7AWO7jbL11MZy926W6MArXYuJ2AqRY2sPfDA1pT6adqhjt"
ACCESS_TOKEN = "851770460455014400-LBj7grHasYzTMjPyysaxTCLBMsnCYMm"
ACCESS_TOKEN_SECRET = "G3ZThMZXaMsznNyQUPkVfN3l8IZB05bckxeTORSNjwrur"

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api1 = API(auth)

In [17]:
CONSUMER_KEY = ""
CONSUMER_SECRET = "KKiNtI8403O6R7MXUowWfM2mGB71eLJX2jeIMsgjGQ5SJrMaDl"
ACCESS_TOKEN = "20743-PbvM6FZjT2LoDSKTfAUpWwSwLKwPrXj25VVyIe5s3mya"
ACCESS_TOKEN_SECRET = "FdqcOey0FdwIhFhTyIuCJOFXwjFOX1EIDHG5vojPq3W51"

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api2 = API(auth)

In [23]:
CONSUMER_KEY = ""
CONSUMER_SECRET = "U7U9dyOrwyIEUZ8Gg6E3umDg2aYxE32Wxe68AmrYVoCpLdpHSE"
ACCESS_TOKEN = "851777049371049984-ItoRu5hrw1c6Q0xfWNAHIm87VpylhBi"
ACCESS_TOKEN_SECRET = "r11r1kkV9J6XhfbJabYdDSVmMfHr588QgiudTx0mGmt9A"

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api3 = API(auth)

In [45]:
CONSUMER_KEY = ""
CONSUMER_SECRET = "UnTNdH6L64vuZ7Am9yfOy6l9tUsJ3MAe9HPlikjzTPd7wb29m2"
ACCESS_TOKEN = "851782771332837376-1oXxwDyBJ3ZNum6t2exwo5DwOtCMLwv"
ACCESS_TOKEN_SECRET = "am9SmXn8jzH82Wkk2GpJ5GbF0CThQcilg5KU3rjy6DZab"

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api4 = API(auth)

In [69]:
CONSUMER_KEY = ""
CONSUMER_SECRET = "v0dqE3aZwZJtMTiHgmqj1OqdjT6VlscdKZR5CbcA68IVzoByu8"
ACCESS_TOKEN = "851789776894210048-ANs4nyLcUSMxlkBeD0AdkpX4WVwufsb"
ACCESS_TOKEN_SECRET = "g6l8V8pWTUGDVCubakBoNxo88guXBZJDd0EJcjzMXNlER"

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api5 = API(auth)

In [82]:
CONSUMER_KEY = ""
CONSUMER_SECRET = "9tUnj3EdrusBXeHsziOf3UibwMViQUlEVW9epMnknbtgjeQ1BN"
ACCESS_TOKEN = "851794545717149696-8Ml5QHfqwOyQgwiJBiZd5rjumTGwPbq"
ACCESS_TOKEN_SECRET = "rmXCc8BZWqQp6ZJOJP7VG2pAerImIi76b60pYM6vmpfh0"

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api6 = API(auth)

Here we use the cursor and try pulling from a  fixed date.

In [121]:
from tweepy import Cursor

query = '#SusanRice'

ids = []
texts = []
times  = []
retweets = []
screen_names = []
followers_counts = []
friends_counts = []

# Only iterate through the first 3 pages
for page in Cursor(api1.search, q=query, result_type='recent', count=100, until="2017-04-04").pages(170):
    
    for tweet in page:

        ids.append(tweet.id)
        texts.append(tweet.text)
        times.append(tweet.created_at)
        retweets.append(tweet.retweet_count)
        screen_names.append(tweet.user.screen_name)
        friends_counts.append(tweet.user.friends_count)
        followers_counts.append(tweet.user.followers_count)

In [125]:
from pandas import DataFrame
df = DataFrame({"id":ids,"text":texts,"time":times,"retweet":retweets,"screen_name":screen_names,"friends_count":friends_counts,"followers_count":followers_counts})
df.tail()

,followers_count,friends_count,id,retweet,screen_name,text,time
16985,288,143,849038793621590017,483,Robo_Elk,"RT @Cernovich: Hey @EliLake, check the time st...",2017-04-03 23:19:57
16986,515,197,849038792761769985,2,TroyLynch8,RT @VonWally: https://t.co/p7mjqjm91k #susanri...,2017-04-03 23:19:57
16987,1234,2502,849038791893549056,37,carenharkins,RT @BinsackSb: #POWWW!! #Confirmed: #SusanRice...,2017-04-03 23:19:57
16988,1678,1693,849038790471692289,2906,crawfishaka,RT @comermd: Will #SusanRice bring down #obama...,2017-04-03 23:19:56
16989,1729,2721,849038790337363968,18,attackteam1,RT @FrogBallerina: Look Dems this is your lega...,2017-04-03 23:19:56


And then we work our way backward, taking the oldest tweet in the group we pulled and saying to get another set of pages, organized so that this index is the upper bound for our search.

In [24]:
for page in Cursor(api2.search, q=query, result_type='recent', count=100, max_id="849038790337363968").pages(170):
    
    for tweet in page:

        ids.append(tweet.id)
        texts.append(tweet.text)
        times.append(tweet.created_at)
        retweets.append(tweet.retweet_count)
        screen_names.append(tweet.user.screen_name)
        friends_counts.append(tweet.user.friends_count)
        followers_counts.append(tweet.user.followers_count)

In [28]:
from pandas import DataFrame
df = DataFrame({"id":ids,"text":texts,"time":times,"retweet":retweets,"screen_name":screen_names,"friends_count":friends_counts,"followers_count":followers_counts})
df.tail()

,followers_count,friends_count,id,retweet,screen_name,text,time
33980,2527,2986,849029060999213056,2084,HearUsNowUSA1,RT @_Makada_: The left-wing propaganda media i...,2017-04-03 22:41:16
33981,440,404,849029060885983232,390,MsFleurs87,"RT @TheRickyDavila: Let's be clear, the WH usi...",2017-04-03 22:41:16
33982,1252,2490,849029060302974978,20,Elizabe05716450,RT @ispyradio: It's all fun and games… until s...,2017-04-03 22:41:16
33983,791,1057,849029059761909760,149,Steve4721076,RT @trump2016fan: #SUSANRICE Who are Muslim a...,2017-04-03 22:41:16
33984,826,1420,849029059170492416,390,DarkLab10,"RT @TheRickyDavila: Let's be clear, the WH usi...",2017-04-03 22:41:16


... and we just keep doing this.

In [29]:
for page in Cursor(api3.search, q=query, result_type='recent', count=100, max_id="849029059170492416").pages(170):
    
    for tweet in page:

        ids.append(tweet.id)
        texts.append(tweet.text)
        times.append(tweet.created_at)
        retweets.append(tweet.retweet_count)
        screen_names.append(tweet.user.screen_name)
        friends_counts.append(tweet.user.friends_count)
        followers_counts.append(tweet.user.followers_count)

Eventually this gets old and weput things in a loop. We'd ideally build an outer loop as well to skip back more than 6 times. 

In [103]:
apis = [api1,api2,api3,api4,api5,api6]

for i in range(6):
    
    mid = min(ids)
    api = apis[i]
    
    print i, min(times)
    
    for page in Cursor(api.search, q=query, result_type='recent', count=100, max_id=str(mid)).pages(170):
    
        for tweet in page:

            ids.append(tweet.id)
            texts.append(tweet.text)
            times.append(tweet.created_at)
            retweets.append(tweet.retweet_count)
            screen_names.append(tweet.user.screen_name)
            friends_counts.append(tweet.user.friends_count)
            followers_counts.append(tweet.user.followers_count)
    
            out = open("tweets/"+str(tweet.id)+".json","wb")
            out.write(dumps(tweet._json))
            out.close()


0 2017-04-02 03:05:22
1 2017-04-02 03:05:22
2 2017-04-02 03:05:22
3 2017-04-02 03:05:22
4 2017-04-02 03:05:22
5 2017-04-02 03:05:22
